<a href="https://colab.research.google.com/github/RaulOAlves/Classifier_DeepLearning_ArcgisPRO/blob/main/Classifier_DeepLearning_ArcgisPRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando as bibliotecas

import arcpy
from arcpy import env
import os
from arcpy.ia import *
from arcgis.learn import *
env.overwriteOutput = True

In [ ]:
# Diretórios de trabalho

# Diretório GDB
workspace1 = r'E:\Cursos\GeoAI\GeoAI_03\GeoAI_03.gdb'

# Diretório raiz
workspace2 = r'E:\Cursos\GeoAI\GeoAI_03'

In [ ]:
# Diretório de Imagens

imagem_dir = r'E:\Cursos\GeoAI\GeoAI_02\Imagens'

# Bandas

b0 = os.path.join(imagem_dir, 'CBERS_4A_WPM_20210727_211_115_L4_BAND0.tif')
b1 = os.path.join(imagem_dir, 'CBERS_4A_WPM_20210727_211_115_L4_BAND1.tif')
b2 = os.path.join(imagem_dir, 'CBERS_4A_WPM_20210727_211_115_L4_BAND2.tif')
b3 = os.path.join(imagem_dir, 'CBERS_4A_WPM_20210727_211_115_L4_BAND3.tif')
b4 = os.path.join(imagem_dir, 'CBERS_4A_WPM_20210727_211_115_L4_BAND4.tif')

# Composição Colorida

In [ ]:
# Diratório da composição

composicao_output = os.path.join(workspace2, 'composicao.tif')


In [ ]:
# Bandas para composição

composicao = r'{};{};{};{}'.format(b4,b2,b3,b1)

In [ ]:
# Composição das bandas

arcpy.management.CompositeBands(in_rasters = composicao,
                                out_raster = composicao_output)

# Pansharpene

In [ ]:
# Diretório do arquivo pansharpene

pan_output = os.path.join(workspace2, 'pansharpene.tif')


In [ ]:
# Realizar o Pansharnepe

arcpy.ia.Pansharpen(b0, composicao_output,True,'Gram-Schmidt').save(pan_output)

# Clip

In [ ]:
# Diretório do clip

# Área de treino
clip_training = os.path.join (workspace2, 'clip_training.tif')
ROI_training = os.path.join (workspace1, 'ROI_training')

# Área de teste
clip_test = os.path.join (workspace2, 'clip_test.tif')
ROI_test = os.path.join (workspace1, 'ROI_test')

In [ ]:
# Função para fazer o clip das área de interesse

def clip_ROI (clip, ROI):
  arcpy.Clip_management(pan_output, '',
                      clip, ROI, '0', 'ClippingGeometry',
                      'MAINTAIN_EXTENT')

In [ ]:
# Roodando a função clip

clip_ROI (clip_training, ROI_training)
clip_ROI (ROI_training, ROI_test)

# Classificando

In [ ]:
# Diretório das amostras e do esquema

amostra_pixel = os.path.join(workspace2, 'amostras_GeoAI.shp')
output_ecd = os.path.join(workspace2, "Class_GeoAI.ecd")

In [ ]:
# Treino e classificação por SVM

arcpy.ia.TrainSupportVectorMachineClassifier(clip_training, amostra_pixel, output_ecd, '',
                                            20, None, 'Classvalue')

In [ ]:
# Diretório de classificação

classificado = os.path.join(workspace2, 'classified.tif')

In [ ]:
# Classificação das imagens

out_raster_dataset = arcpy.ia.ClassifyRaster(clip_training, output_ecd, ''); out_raster_dataset.save(classificado)

# Transformar raster to polygon

In [ ]:
# Diretório dos polígonos

out_poligon = os.path.join(workspace1, 'poligons')
classificada = os.path.join(workspace2, 'classified.tif')

In [ ]:
# Raster to Polygon

arcpy.conversion.RasterToPolygon(classificada,
                                out_poligon,
                                'SIMPLIFY',
                                'Classvalue',
                                'SINGLE_OUTER_PART',
                                None)

# Imagem para Classificar

In [ ]:
# Diretório das imagem para classificar e amostras

input_imagem = os.path.join(workspace2, 'clip_training.tif')
out_amostras = os.path.join(workspace2, 'amostra')

In [ ]:
# Classificação da imagem

arcpy.ia.ExportTrainingDataForDeepLearning(input_imagem,
                                          out_amostras,
                                          out_poligon,
                                          'TIFF',
                                          256,256,128,128,
                                          'ONLY_TILES_WITH_FEATURES',
                                          'Classified_Tiles',
                                          0, 'gridcode',
                                          0, None, 0,
                                          'MAP_SPACE',
                                          'PROCESS_AS_MOSAICKED_IMAGE',
                                          'NO_BLACKEN',
                                          'FIXED_SIZE',
                                          None)

# Treinando o Modelo

In [ ]:
# Ler as amostras

data = prepare_data(path = out_amostras,
                   batch_size=3,
                   dataset_type='Classified_Tiles',
                   chip_size= 256)

In [ ]:
# Configurando o modelos

model = UnetClassifier(data, backbone = 'resnet34')

In [ ]:
# Learning Rate

lr = 0.001

In [ ]:
# Treinamento do modelo em 40 épocas

model.fit(40, lr=lr)

In [ ]:
# Visualização das métricas de Precision, Recall, F1 e IOU

metrics = model.per_class_metrics()
metrics
model.mIOU()

In [ ]:
# Resultado visual do treinamento

model.show_results()

In [ ]:
# Salvar o modelo

model.save('model_UNET')

# Classificando uma nova Imagem

In [ ]:
# Configuração de classificação

# Diretórios para classificação
nova_imagem = os.path.join (workspace2, 'clip_test.tif')
modelo = os.path.join (workspace2, 'amostra\models\model_UNET\model_UNET.dlpk')
out_new_classified = os.path.join(workspace1, 'imagem_classify')

# Definição dos parâmetros de classificação
parametros = 'padding 56; batch_size 4; predict_background True; tile_size 256'

In [ ]:
# Classificação da nova imagem

with arcpy.EnvManager (processorType='GPU'):
    out_classified_raster = arcpy.ia.ClassifyPixelsUsingDeepLearning(nova_imagem,
                                                                    modelo,
                                                                    parametros,
                                                                    'PROCESS_AS_MOSAICKED_IMAGE',
                                                                    None); out_classified_raster.save(out_new_classified)